<a href="https://colab.research.google.com/github/NeelKanwal/BERTOLOGY-Based-Extractive-Summarization-for-Clinical-Notes/blob/master/FineTunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# just run this once to upgrade the RAM for Colab
a = []
while(1):
    a.append('1')

In [1]:
!git clone https://github.com/NVIDIA/apex.git && cd apex && python setup.py install --cuda_ext --cpp_ext

Cloning into 'apex'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 6777 (delta 17), reused 8 (delta 5), pack-reused 6745
Receiving objects: 100% (6777/6777), 13.76 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (4523/4523), done.
torch.__version__  =  1.5.0+cu101
setup.py:46: UserWarning: Option --pyprof not specified. Not installing PyProf dependencies!
  warnings.warn("Option --pyprof not specified. Not installing PyProf dependencies!")

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writi

In [2]:
!pip install transformers
!pip install tensorboardx
!pip install seqeval
!pip install simpletransformers
!pip install jsonlines

In [3]:
!pip install apex

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import torch
device = torch.cuda.current_device()
device_name = torch.cuda.get_device_name(device)
cuda_flag = torch.cuda.is_available()

print('current device='+device_name)

current device=Tesla K80


In [6]:
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [7]:
import os
os.chdir("/content/drive/My Drive/clinical_notes_ICD9")
%ls

 apex/                                  mimicdata/
 cache_dir/                             models.py
 clinicalNLP.yml                        notebooks/
 constants.py                           outputs/
 converge.py                            persistence.py
 dataproc/                              predictions/
 dataproc_mimic_III.ipynb               preprocess.py
 datasets.py                            __pycache__/
 evaluation.py                          README.md
 get_metrics_for_saved_predictions.py   runs/
 ICD9_descriptions                      saved_models/
 input_files/                           training_2.py
 learn/                                'training command.txt'
 LICENSE                                training.py


In [0]:
import torch
from torch import optim
from torch.utils.data import DataLoader
import datasets
from datasets import MimicDataset
from datasets import collate

In [0]:
import argparse
import pandas as pd

FILES ="/content/drive/My Drive/clinical_notes_ICD9/input_files/"

args = argparse.Namespace()

args.data_path = FILES + 'notes_labeled_train.ndjson'
args.Y="full"
args.vocab = FILES + 'vocab.csv'
args.data_dir = '/content/drive/My Drive/data'
args.embed_desc = True
args.exclude_non_billable = False
args.include_invalid = False
args.max_len=3000

In [0]:
dicts = datasets.load_lookups(args)

In [11]:
num_labels_fine = len(dicts['ind2c'])
num_labels_coarse = len(dicts['ind2c_coarse'])
print(num_labels_fine)
print(num_labels_coarse)

8924
1167


In [12]:
dataset_train = MimicDataset(args.data_path, dicts, num_labels_fine, num_labels_coarse, args.max_len) # data_id,target_fine,target_coarse,hadm_id,doc
dataset_dev = MimicDataset(args.data_path.replace('train', 'dev'), dicts, num_labels_fine, num_labels_coarse, args.max_len)

loading data from /content/drive/My Drive/clinical_notes_ICD9/input_files/notes_labeled_train.ndjson
loading data from /content/drive/My Drive/clinical_notes_ICD9/input_files/notes_labeled_dev.ndjson


In [0]:
import pandas as pd
import numpy as np
train=[data for data in dataset_train]
train=[[" ".join(data[4]),data[2].cpu().detach().numpy()] for data in train]
test=[data for data in dataset_dev]
test=[[" ".join(data[4]),data[2].cpu().detach().numpy()] for data in test]

In [14]:
df_test=pd.DataFrame(data=test,columns=["text","label"])
df_test["label"] = df_test["label"].apply(lambda val: val.astype(int))
df_test.head()

,text,label
0,admission date december discharge date decembe...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,admission date may discharge date may date of ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,admission date july discharge date july date o...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,admission date april discharge date may date o...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,admission date june discharge date june date o...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
df_train=pd.DataFrame(data=train,columns=["text","label"])
df_train["label"] = df_train["label"].apply(lambda val: val.astype(int))
df_train.head()

,text,label
0,admission date october discharge date october ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,admission date march discharge date march date...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,admission date may discharge date june date of...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,admission date november discharge date novembe...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,admission date june discharge date july date o...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [0]:
args = {
  "output_dir": "/content/outputs/",
  "cache_dir": "/content/cache/",

  "fp16": False,
  "fp16_opt_level": "O1",
  "max_seq_length": 2048,
  "train_batch_size": 8,
  "eval_batch_size": 8,
  "gradient_accumulation_steps": 1,
  "num_train_epochs": 3,
  "weight_decay": 0,
  "learning_rate": 3e-5,
  "adam_epsilon": 1e-8,
  "warmup_ratio": 0.06,
  "warmup_steps": 0,
  "max_grad_norm": 1.0,
  "do_lower_case": False,

  "logging_steps": 50,
  "evaluate_during_training": False,
  "evaluate_during_training_steps": 2000,
  "evaluate_during_training_verbose": False,
  "use_cached_eval_features": True,
  "save_eval_checkpoints": True,
  "save_steps": 2000,
  "no_cache": False,
  "save_model_every_epoch": True,
  "tensorboard_dir": None,

  "overwrite_output_dir": False,
  "reprocess_input_data": False,

  "process_count": 1,
  "n_gpu": 1,
  "silent": False,
  "use_multiprocessing": True,

  "wandb_project": None,
  "wandb_kwargs": {},

  "use_early_stopping": True,
  "early_stopping_patience": 3,
  "early_stopping_delta": 0,

  "manual_seed": None,
}

In [0]:
# https://pypi.org/project/simpletransformers/#classificationmodel
from simpletransformers.classification import MultiLabelClassificationModel
model = MultiLabelClassificationModel('bert', 'bert-base-uncased', num_labels=num_labels_coarse,use_cuda=cuda_flag ,args=args)

In [0]:
# Train the model
from datetime import datetime
start=datetime.now()
model.train_model(df_train,show_running_loss=True)
stop = datetime.now()
print("Time Elapsed: ",stop - start)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:256: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Features loaded from cache at cache_dir/cached_train_bert_128_1167_47723


Running loss: 0.695532

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.044585

Running loss: 0.029490

Running loss: 0.020265
Training of bert model complete. Saved to outputs/.


In [0]:
# Evaluate the model
# The default evaluation metric used is Label Ranking Average Precision (LRAP) Score.
import sklearn
result, model_outputs, predictions = model.eval_model(df_test,verbose=True,acc=sklearn.metrics.accuracy_score)

In [0]:
# predictions, raw_outputs = model.predict(["Some arbitary sentence"])
len(model_outputs[0])

1167